In [39]:
import requests
import pandas as pd
import scipy.spatial.distance as ssd

In [0]:
def query(postcode, offset=0):
  URL = 'https://api.ordnancesurvey.co.uk/places/v1/addresses/postcode'
  params = {'postcode': postcode, 'key': 'dxp6QRGemG64Aj4HdswNnIAzjIyKwMt1', 'offset': offset}
  data = requests.get(URL, params=params).json()
  header = data['header']
  results = data['results']
  yield [d['DPA'] for d in results]
  if header['totalresults'] > offset + header['maxresults']:
    yield from query(postcode, offset=offset + header['maxresults'])

In [25]:
from google.colab import files
uploaded = files.upload()

Saving PTALs.csv to PTALs.csv


In [30]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

ModuleNotFoundError: ignored

In [52]:
#from io import BytesIO
#ptal = pd.read_csv(BytesIO(uploaded['PTALs.csv']))
#points = ptal[['X', 'Y']].values

directory = input()
datapath= 'PTAL_Grid_2018_2021_2031.csv'
ptal_data = pd.read_csv(directory + datapath)
ptal_data.head()

ptal_data = ptal_data.rename(columns= {'X': 'X_COORDINATE', \
                                       'Y': 'Y_COORDINATE', \
                                       'PTALBY': 'PTAL_2018'})
points = ptal_data[['X_COORDINATE', 'Y_COORDINATE']]
print(points.iloc[:10, :])

/Users/vincentmargerin/CityDataHack/data/
   X_COORDINATE  Y_COORDINATE
0        527245        173952
1        527345        173952
2        527445        173952
3        527545        173952
4        527645        173952
5        527745        173952
6        527845        173952
7        527945        173952
8        528045        173952
9        528145        173952


In [53]:
def get_data(postcode):
  df = pd.concat(pd.DataFrame(res) for res in query(postcode))
  df['PTAL_2018'] = df.apply(lambda row: ptal_data.PTAL_2018[ssd.cdist([[row['X_COORDINATE'], row['Y_COORDINATE']]], points).argmin()], axis=1)
  return df

In [54]:
df = get_data('N5')

In [55]:
df.head()

,ADDRESS,BLPU_STATE_CODE,BLPU_STATE_CODE_DESCRIPTION,BLPU_STATE_DATE,BUILDING_NAME,BUILDING_NUMBER,CLASSIFICATION_CODE,CLASSIFICATION_CODE_DESCRIPTION,DEPARTMENT_NAME,DEPENDENT_THOROUGHFARE_NAME,...,RPC,STATUS,SUB_BUILDING_NAME,THOROUGHFARE_NAME,TOPOGRAPHY_LAYER_TOID,UDPRN,UPRN,X_COORDINATE,Y_COORDINATE,PTAL_2018
0,"FLAT 9, MATTHEWS COURT, HIGHBURY GRANGE, LONDO...",NaN,Unknown/Not applicable,NaN,MATTHEWS COURT,NaN,RD,Dwelling,NaN,NaN,...,1,APPROVED,FLAT 9,HIGHBURY GRANGE,osgb1000005904328,15522705,5300043373,532011.0,185698.0,2
1,"FLAT 21, DRAKELEY COURT, AUBERT PARK, LONDON, ...",NaN,Unknown/Not applicable,NaN,DRAKELEY COURT,NaN,RD06,Self Contained Flat (Includes Maisonette / Apa...,NaN,NaN,...,1,APPROVED,FLAT 21,AUBERT PARK,osgb1000005746331,15519501,5300044167,531608.0,185825.0,5
2,"54, PARK VIEW, COLLINS ROAD, LONDON, N5 2UD",NaN,Unknown/Not applicable,NaN,NaN,54,RD06,Self Contained Flat (Includes Maisonette / Apa...,NaN,PARK VIEW,...,1,APPROVED,NaN,COLLINS ROAD,osgb1000005904063,15524288,5300022720,532408.0,185842.0,3
3,"FLAT 11, 71A, DRAYTON PARK, LONDON, N5 1AN",NaN,Unknown/Not applicable,NaN,71A,NaN,RD06,Self Contained Flat (Includes Maisonette / Apa...,NaN,NaN,...,1,APPROVED,FLAT 11,DRAYTON PARK,osgb1000001802755910,50611428,10012793706,531447.0,185570.0,6a
4,"141, PARK VIEW, COLLINS ROAD, LONDON, N5 2UE",NaN,Unknown/Not applicable,NaN,NaN,141,RD06,Self Contained Flat (Includes Maisonette / Apa...,NaN,PARK VIEW,...,1,APPROVED,NaN,COLLINS ROAD,osgb1000005904047,15524363,5300022795,532365.0,185905.0,2


In [22]:
df.CLASSIFICATION_CODE_DESCRIPTION.value_counts()

Dwelling                                                           4748
Self Contained Flat (Includes Maisonette / Apartment)              3405
Property Shell                                                     1250
Terraced                                                            203
Shop / Showroom                                                     132
Dual Use                                                             47
Commercial                                                           40
Retail                                                               31
PO Box                                                               28
Office / Work Studio                                                 24
Detached                                                             17
Additional Mail / Packet Addressee                                   17
Semi-Detached                                                        11
Residential                                                     

In [0]:
EC1 = get_data('EC1')

In [0]:
EC1['ptal'] = EC1.apply(lambda row: ptal.PTALBY[cdist([[row['X_COORDINATE'], row['Y_COORDINATE']]], points).argmin()], axis=1)

In [52]:
EC1

,ADDRESS,BLPU_STATE_CODE,BLPU_STATE_CODE_DESCRIPTION,BLPU_STATE_DATE,BUILDING_NAME,BUILDING_NUMBER,CLASSIFICATION_CODE,CLASSIFICATION_CODE_DESCRIPTION,DEPARTMENT_NAME,DEPENDENT_THOROUGHFARE_NAME,...,RPC,STATUS,SUB_BUILDING_NAME,THOROUGHFARE_NAME,TOPOGRAPHY_LAYER_TOID,UDPRN,UPRN,X_COORDINATE,Y_COORDINATE,ptal
0,"FLAT 6, 169, CENTRAL STREET, LONDON, EC1V 8BS",2,In use,17/10/2016,NaN,169,RD06,Self Contained Flat (Includes Maisonette / Apa...,NaN,NaN,...,1,APPROVED,FLAT 6,CENTRAL STREET,osgb1000005893651,52774613,10090550510,532083.0,182770.0,3
1,"APARTMENT 610, 10, HOSIER LANE, LONDON, EC1A 9LS",2,In use,23/04/2009,NaN,10,RD06,Self Contained Flat (Includes Maisonette / Apa...,NaN,NaN,...,2,APPROVED,APARTMENT 610,HOSIER LANE,osgb1000001802607666,27894775,95505886,531738.0,181564.0,6b
2,"APARTMENT 603, 9B, CLERKENWELL ROAD, LONDON, E...",NaN,Unknown/Not applicable,NaN,9B,NaN,RD06,Self Contained Flat (Includes Maisonette / Apa...,NaN,NaN,...,1,APPROVED,APARTMENT 603,CLERKENWELL ROAD,osgb1000001802772612,28608141,10012790424,531990.0,182121.0,6b
3,"FLAT 2, 390, CITY ROAD, LONDON, EC1V 2QA",NaN,Unknown/Not applicable,NaN,NaN,390,RD06,Self Contained Flat (Includes Maisonette / Apa...,NaN,NaN,...,1,APPROVED,FLAT 2,CITY ROAD,osgb1000005740418,28758728,10012791212,531614.0,183062.0,6a
4,"FLAT 61, CHADWORTH HOUSE, LEVER STREET, LONDON...",NaN,Unknown/Not applicable,NaN,CHADWORTH HOUSE,NaN,RD,Dwelling,NaN,NaN,...,1,APPROVED,FLAT 61,LEVER STREET,osgb1000005893782,8176069,5300056347,532182.0,182600.0,6a
5,"FLAT 20, TUNBRIDGE HOUSE, ST. JOHN STREET, LON...",NaN,Unknown/Not applicable,NaN,TUNBRIDGE HOUSE,NaN,RD,Dwelling,NaN,NaN,...,1,APPROVED,FLAT 20,ST. JOHN STREET,osgb1000005734343,8173167,5300085192,531540.0,182761.0,6b
6,"FLAT 46, COLTASH COURT, 152, WHITECROSS STREET...",NaN,Unknown/Not applicable,NaN,COLTASH COURT,152,RD,Dwelling,NaN,NaN,...,1,APPROVED,FLAT 46,WHITECROSS STREET,osgb1000005894942,8182649,5300097655,532358.0,182332.0,6a
7,"FLAT 8, CITY VIEW APARTMENTS 29A, SAFFRON HILL...",NaN,Unknown/Not applicable,NaN,CITY VIEW APARTMENTS 29A,NaN,RD,Dwelling,NaN,NaN,...,1,APPROVED,FLAT 8,SAFFRON HILL,osgb1000005732747,51216719,5152471,531478.0,181830.0,6b
8,"CLERKENWELL FILMS, 82-84, CLERKENWELL ROAD, LO...",NaN,Unknown/Not applicable,NaN,82-84,NaN,CO01,Office / Work Studio,NaN,NaN,...,1,HISTORICAL,NaN,CLERKENWELL ROAD,osgb1000005735911,50384941,10090549337,531737.0,182124.0,6a
9,"FLAT 81, GREAT ARTHUR HOUSE, GOLDEN LANE ESTAT...",2,In use,23/04/2009,GREAT ARTHUR HOUSE,NaN,RD06,Self Contained Flat (Includes Maisonette / Apa...,NaN,NaN,...,1,APPROVED,FLAT 81,GOLDEN LANE ESTATE,osgb1000005894760,8181068,200000071776,532193.0,182074.0,6b
